In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import AdamW
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import log_loss
from tensorflow.keras.applications import ResNet50

# Проверяем доступность GPU
print("Доступные устройства:", tf.config.list_physical_devices('GPU'))

# Включаем смешанную точность для ускорения вычислений
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

# Пути к данным
train_dir = "data/train_sample"
val_dir = "data/test_sample"

# Получаем список всех изображений
train_images = [os.path.join(train_dir, img) for img in os.listdir(train_dir) if img.endswith(('.jpg', '.png'))]
val_images = [os.path.join(val_dir, img) for img in os.listdir(val_dir) if img.endswith(('.jpg', '.png'))]

# Проверяем наличие изображений
assert len(train_images) > 0, "Ошибка: нет изображений в train_sample"
assert len(val_images) > 0, "Ошибка: нет изображений в test_sample"

# Функция загрузки изображений и меток
def load_images(image_paths):
    images = []
    labels = []
    for img_path in image_paths:
        img = cv2.imread(img_path)
        if img is None:
            print(f"Предупреждение: изображение {img_path} не загружено")
            continue
        img = cv2.resize(img, (256, 256))
        img = img / 255.0
        images.append(img)

        label = 1 if "dog" in img_path.lower() else 0
        labels.append(label)

    return np.array(images), np.array(labels)

# Загружаем тренировочные и валидационные данные
X_train, y_train = load_images(train_images)
X_val, y_val = load_images(val_images)

# Разделяем данные на обучение и тест
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Аугментация данных
datagen = ImageDataGenerator(rotation_range=40, width_shift_range=0.2, height_shift_range=0.2,
                             shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

# Используем предобученную ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
base_model.trainable = True  

# Замораживаем первые 2 слоя
for layer in base_model.layers[:2]:
    layer.trainable = False

# Создаём модель
model = Sequential([base_model, Flatten(), Dense(512, activation='swish'),
                    BatchNormalization(), Dropout(0.5), Dense(256, activation='swish'),
                    BatchNormalization(), Dropout(0.3), Dense(1, activation='sigmoid')])

# Компиляция модели
model.compile(optimizer=AdamW(learning_rate=0.00003),
              loss=tf.keras.losses.LogCosh(), metrics=['accuracy'])

# Обучение модели на GPU
with tf.device('/GPU:0'):
    history = model.fit(datagen.flow(X_train, y_train, batch_size=32),
                        epochs=75, validation_data=(X_val, y_val))

# Сохранение модели
model.save('cats_vs_dogs_resnet_gpu.h5')

# Проверка Log Loss
y_pred = model.predict(X_test)
logloss = log_loss(y_test, y_pred)
print(f"Log Loss модели: {logloss:.4f}")

Доступные устройства: []


d:\Documents\python_projects\Netology\Компьютерное зрение\.venv\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


MemoryError: Unable to allocate 24.0 MiB for an array with shape (32, 256, 256, 3) and data type float32